# **Data Wrangling**

## Dataset Used

The dataset used in this notebook comes from IBM Cloud: https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/VYPrOu0Vs3I0hKLLjiPGrA/survey-data-with-duplicate.csv

> Clicking on the link will download a CSV file



## Objectives

* Identify duplicate rows in the dataset
* Remove duplicate rows and verify the removal
* Find columns with missing values
* Impute the missing values
* Perform Data Standardizing and Normalizing for certain columns

## Download and Import Libraries

In [ ]:
# Run this cell if the libraries are not installed yet
%pip install pandas
%pip install numpy

In [1]:
# Importing the pandas and numpy libraries
import pandas as pd
import numpy as np

## Loading the Data

In [3]:
# Read the CSV file into a dataframe
url = 'https://cf-courses-data.s3.us.cloud-object-storage.appdomain.cloud/VYPrOu0Vs3I0hKLLjiPGrA/survey-data-with-duplicate.csv'
data = pd.read_csv(url)

# Options to display all rows and columns
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

# Display the first five rows
data.head()

,ResponseId,MainBranch,Age,Employment,RemoteWork,Check,CodingActivities,EdLevel,LearnCode,LearnCodeOnline,TechDoc,YearsCode,YearsCodePro,DevType,OrgSize,PurchaseInfluence,BuyNewTool,BuildvsBuy,TechEndorse,Country,Currency,CompTotal,LanguageHaveWorkedWith,LanguageWantToWorkWith,LanguageAdmired,DatabaseHaveWorkedWith,DatabaseWantToWorkWith,DatabaseAdmired,PlatformHaveWorkedWith,PlatformWantToWorkWith,PlatformAdmired,WebframeHaveWorkedWith,WebframeWantToWorkWith,WebframeAdmired,EmbeddedHaveWorkedWith,EmbeddedWantToWorkWith,EmbeddedAdmired,MiscTechHaveWorkedWith,MiscTechWantToWorkWith,MiscTechAdmired,ToolsTechHaveWorkedWith,ToolsTechWantToWorkWith,ToolsTechAdmired,NEWCollabToolsHaveWorkedWith,NEWCollabToolsWantToWorkWith,NEWCollabToolsAdmired,OpSysPersonal use,OpSysProfessional use,OfficeStackAsyncHaveWorkedWith,OfficeStackAsyncWantToWorkWith,OfficeStackAsyncAdmired,OfficeStackSyncHaveWorkedWith,OfficeStackSyncWantToWorkWith,OfficeStackSyncAdmired,AISearchDevHaveWorkedWith,AISearchDevWantToWorkWith,AISearchDevAdmired,NEWSOSites,SOVisitFreq,SOAccount,SOPartFreq,SOHow,SOComm,AISelect,AISent,AIBen,AIAcc,AIComplex,AIToolCurrently Using,AIToolInterested in Using,AIToolNot interested in Using,AINextMuch more integrated,AINextNo change,AINextMore integrated,AINextLess integrated,AINextMuch less integrated,AIThreat,AIEthics,AIChallenges,TBranch,ICorPM,WorkExp,Knowledge_1,Knowledge_2,Knowledge_3,Knowledge_4,Knowledge_5,Knowledge_6,Knowledge_7,Knowledge_8,Knowledge_9,Frequency_1,Frequency_2,Frequency_3,TimeSearching,TimeAnswering,Frustration,ProfessionalTech,ProfessionalCloud,ProfessionalQuestion,Industry,JobSatPoints_1,JobSatPoints_4,JobSatPoints_5,JobSatPoints_6,JobSatPoints_7,JobSatPoints_8,JobSatPoints_9,JobSatPoints_10,JobSatPoints_11,SurveyLength,SurveyEase,ConvertedCompYearly,JobSat
0,1,I am a developer by profession,Under 18 years old,"Employed, full-time",Remote,Apples,Hobby,Primary/elementary school,Books / Physical media,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,United States of America,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,I have never visited Stack Overflow or the Sta...,NaN,NaN,NaN,NaN,NaN,Yes,Very favorable,Increase productivity,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,I am a developer by profession,35-44 years old,"Employed, full-time",Remote,Apples,Hobby;Contribute to open-source projects;Other...,"Bachelor’s degree (B.A., B.S., B.Eng., etc.)",Books / Physical media;Colleague;On the job tr...,Technical documentation;Blogs;Books;Written Tu...,API document(s) and/or SDK document(s);User gu...,20,17,"Developer, full-stack",NaN,NaN,NaN,NaN,NaN,United Kingdom of Great Britain and Northern I...,NaN,NaN,Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS...,Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS...,Bash/Shell (all shells);Go;HTML/CSS;Java;JavaS...,Dynamodb;MongoDB;PostgreSQL,PostgreSQL,PostgreSQL,Amazon Web Services (AWS);Heroku;Netlify,Amazon Web Services (AWS);Heroku;Netlify,Amazon Web Services (AWS);Heroku;Netlify,Express;Next.js;Node.js;React,Express;Htmx;Node.js;React;Remix,Express;Node.js;React,NaN,NaN,NaN,NaN,NaN,NaN,Docker;Homebrew;Kubernetes;npm;Vite;Webpack,Docker;Homebrew;Kubernetes;npm;Vite;Webpack,Docker;Homebrew;Kubernetes;npm;Vite;Webpack,PyCharm;Visual Studio Code;WebStorm,PyCharm;Visual Studio Code;WebStorm,PyCharm;Visual Studio Code;WebStorm,MacOS;Windows,MacOS,NaN,NaN,NaN,Microsoft Teams;Slack,Slack,Slack,NaN,NaN,NaN,Stack Overflow for Teams (private knowledge sh...,Multiple times per day,Yes,Multiple times per day,Quickly finding code solutions;Finding reliabl...,"Yes, definitely","No, and I don't plan to",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Yes,Individual contributor,17.0,Agree,Disagree,Agree,Agree,Agree,Neither agree nor disa

## Handling Duplicate Rows

### Finding Duplicates

In [3]:
# Finding the number of duplicate rows
dup_rows = data[data.duplicated()]
num_dups = dup_rows.shape[0]
print(f'There are {num_dups} duplicate rows')

There are 20 duplicate rows


### Removing Duplicates

In [4]:
# Removing the duplicate rows and verifying
df = data.drop_duplicates()
num_dups = df[df.duplicated()].shape[0]
print(f'There are {num_dups} duplicate rows')

There are 0 duplicate rows


## Handling Missing Values

### Finding Missing Values

In [ ]:
# This method displays the non-null counts for each column
# The total rows is 65437 
# Every column with less than the total row count means it has missing values 
df.info(verbose=True, show_counts=True)

<class 'pandas.core.frame.DataFrame'>
Index: 65437 entries, 0 to 65436
Data columns (total 114 columns):
 #    Column                          Non-Null Count  Dtype  
---   ------                          --------------  -----  
 0    ResponseId                      65437 non-null  int64  
 1    MainBranch                      65437 non-null  object 
 2    Age                             65437 non-null  object 
 3    Employment                      65437 non-null  object 
 4    RemoteWork                      54806 non-null  object 
 5    Check                           65437 non-null  object 
 6    CodingActivities                54466 non-null  object 
 7    EdLevel                         60784 non-null  object 
 8    LearnCode                       60488 non-null  object 
 9    LearnCodeOnline                 49237 non-null  object 
 10   TechDoc                         40897 non-null  object 
 11   YearsCode                       59869 non-null  object 
 12   YearsCodePro         

In [10]:
# This find the top 5 columns with the most missing values
df.isnull().sum().sort_values(ascending = False).head()

AINextMuch less integrated    64289
AINextLess integrated         63082
AINextNo change               52939
AINextMuch more integrated    51999
EmbeddedAdmired               48704
dtype: int64

### Imputing Missing Values in Numeric Columns

#### ConvertedCompYearly Column

In [ ]:
# Method 1 - fillna

# Calculating the average value rounded to 0 decimals
average_comp = round(df['ConvertedCompYearly'].mean(), 0)

# Filling in the NaN values with the average
df['ConvertedCompYearly'].fillna(average_comp, inplace = True)

In [15]:
# Verify the imputation
df['ConvertedCompYearly'].isnull().value_counts()

ConvertedCompYearly
False    65437
Name: count, dtype: int64

#### WorkExp Column

In [ ]:
# Method 2 - replace

# Calculating the average value rounded to 0 decimals
avg_work_exp = round(df['WorkExp'].mean(), 0)

# Replacing the NaN values with the average
df['WorkExp'].replace(np.nan, avg_work_exp, inplace = True)

In [17]:
# Verify the imputation
df['WorkExp'].isnull().value_counts()

WorkExp
False    65437
Name: count, dtype: int64

### Imputing Missing Values in Categorical Columns

#### RemoteWork Column

In [ ]:
# Finding the most frequent value
most_remote = df['RemoteWork'].mode()[0]

# Filling the missing values
df['RemoteWork'].fillna(most_remote, inplace = True)

In [19]:
# Verify the imputation
df['RemoteWork'].isnull().value_counts()

RemoteWork
False    65437
Name: count, dtype: int64

#### SurveyEase Column

In [ ]:
# Finding the most frequent value
freq_ease = df['SurveyEase'].mode()[0]

# Replace the NaN with most frequent value
df['SurveyEase'].replace(np.nan, freq_ease, inplace = True)

In [21]:
# Verify the imputation
df['SurveyEase'].isnull().value_counts()

SurveyEase
False    65437
Name: count, dtype: int64

## Data Normalization

### Min-Max Scaling

(data - min) / (max - min)

In [ ]:
# Creating a new column to place the normalize values
df['ConvertedCompYearly_MinMax'] = (df['ConvertedCompYearly'] - df['ConvertedCompYearly'].min()) / (df['ConvertedCompYearly'].max() - df['ConvertedCompYearly'].min())

In [28]:
# Compare the normalized and original values
df[['ConvertedCompYearly_MinMax', 'ConvertedCompYearly']].sample(n = 5, random_state = 42)

,ConvertedCompYearly_MinMax,ConvertedCompYearly
36662,0.005300,86155.0
36720,0.005219,84851.0
30718,0.004921,80000.0
17353,0.004625,75184.0
39838,0.020466,332712.0


### Z-score normalization

(data - mean) / standard deviation

In [ ]:
# Placing the normalize values in a new column
df['ConvertedCompYearly_Zscore'] = (df['ConvertedCompYearly'] - df['ConvertedCompYearly'].mean()) / df['ConvertedCompYearly'].std()

In [29]:
# Compare the normalized and original values
df[['ConvertedCompYearly_Zscore', 'ConvertedCompYearly']].sample(n = 5, random_state = 42)

,ConvertedCompYearly_Zscore,ConvertedCompYearly
36662,-9.205112e-07,86155.0
36720,-1.166864e-02,84851.0
30718,-5.507362e-02,80000.0
17353,-9.816544e-02,75184.0
39838,2.206101e+00,332712.0


## Other Techniques

### Binning

Creating a new column: 'ExperienceLevel' based on the 'YearsCodePro' Column

In [ ]:
# Fixing the format of the YearsCodePro column

# Replacing the strings to a number
df['YearsCodePro'].replace('Less than 1 year', '0', inplace = True)
df['YearsCodePro'].replace('More than 50 years', '50', inplace = True)
df['YearsCodePro'] = df['YearsCodePro'].astype(float)

# Replace NaN values
avg_years = round(df['YearsCodePro'].mean(), 0)
df['YearsCodePro'].fillna(avg_years, inplace = True)

In [ ]:
# Create bins of equal length
bins = np.linspace(min(df['YearsCodePro']), max(df['YearsCodePro']), 16)

# Store the bin names 
bin_names = ['Entry', 'Mid', 'Senior', 'Lead', 'Architect', 'Architect', 'Architect', 'Architect', 'Architect', 'Architect', 'Architect', 'Architect', 'Architect', 'Architect', 'Architect']

# Using the function cut to apply the bins
df['ExperienceLevel'] = pd.cut(df['YearsCodePro'], bins, labels = bin_names, include_lowest=True, ordered=False)

In [65]:
# Displaying 10 rows
df[['YearsCodePro', 'ExperienceLevel']].sample(n = 10, random_state = 42)

,YearsCodePro,ExperienceLevel
36662,1.0,Entry
36720,6.0,Mid
30718,5.0,Mid
17353,17.0,Architect
39838,14.0,Architect
6247,11.0,Lead
31793,12.0,Lead
21403,9.0,Senior
57614,15.0,Architect
24728,10.0,Senior


### One-hot Encoding



In [33]:
# Display the values and counts in the MainBranch column
df['MainBranch'].value_counts()

MainBranch
I am a developer by profession                                                           50207
I am not primarily a developer, but I write code sometimes as part of my work/studies     6511
I am learning to code                                                                     3875
I code primarily as a hobby                                                               3334
I used to be a developer by profession, but no longer am                                  1510
Name: count, dtype: int64

In [34]:
# Using the method get_dummies to encode the MainBranch column
# Values only consist of True or False
# Rename the columns for better readability
df_encoded = pd.get_dummies(df['MainBranch'])
df_encoded.columns = ['ProDeveloper', 'Learner', 'OccasionalCoder', 'HobbyCoder', 'FormerDev']
df_encoded.head()

,ProDeveloper,Learner,OccasionalCoder,HobbyCoder,FormerDev
0,True,False,False,False,False
1,True,False,False,False,False
2,True,False,False,False,False
3,False,True,False,False,False
4,True,False,False,False,False


In [35]:
# Adding the new encoded values to the dataframe
new_df = pd.concat([df, df_encoded], axis = 1)